<a href="https://colab.research.google.com/github/kingsgameBen/MLbasic_0223/blob/main/cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


input -> 模型 -> output
深度學習 範例 ： edge cat (圖像翻譯)[https://affinelayer.com/pixsrv/]
-------------------------------------------
input    |  output
-------------------------------------------
         |  cluster(分群 問題)
            :辨認梵谷莫內達文西的作品?沒有output的選擇題：利用「題目的相似度」猜答案
----------------------------------------
刪除ans欄位 -> 沒有答案欄位

最簡單的分群方法
KMeans (K個中心)
1. 隨便取3個中心 (隨機選取三個樣本)
2. 計算距離->歐幾里得距離$\sqrt{\left( Sl_1 - Sl_2 \right)^2 + \left( Sw_1 - Sw_2 \right)^2+\left( Pl_1 - Pl_2 \right)^2 + \left( Pw_1 - Pw_2 \right)^2}$
3. 每個點標注(固定圓心找標註)
   if 真的中心：min   $\sum\left( 平均 - P_i \right)^2$
4. 重選圓心(固定標註找圓心, 直到圓心不再變動為止)
  步驟3、4不斷迭代, 直到標註與圓心都穩定！

[https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html]
*若一開始選的點都相同或太近, 只能分成一群 -> K-means++改善此問題, 一開始取樣遠一點
- tol: tolerance 容許圓心兩次的微幅變動容忍值(收斂範圍)



In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
# 分辨鳶尾花
iris = load_iris() # 3 type iris
df = pd.DataFrame(iris['data']
                  ,columns = iris['feature_names'])
print(iris['feature_names'])
print(iris)
df

In [18]:
from sklearn.cluster import KMeans
cluster = KMeans(n_clusters=3)
cluster.fit(iris["data"])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
cluster.labels_

In [34]:
# 將分群的結果儲存到DataFrame的新欄位
df["label"] = cluster.labels_
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [20]:
# 會找出3群圓心(已知有3群的情況下), 算平均
# 實際不存在這朵花, sepal length, sepal width, petal length, petal width
# 但是Kmeans會找真正的圓心
cluster.cluster_centers_

array([[5.006     , 3.428     , 1.462     , 0.246     ],
       [5.9016129 , 2.7483871 , 4.39354839, 1.43387097],
       [6.85      , 3.07368421, 5.74210526, 2.07105263]])

matplotlib 資料視覺化
進階函式庫-> (seaborn)[https://seaborn.pydata.org/]
[https://seaborn.pydata.org/generated/seaborn.scatterplot.html#seaborn.scatterplot]
- hue: +1 (利用其他方式額外增加一組資訊)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# hue色調:拿取DataFrame的一行[分群後的labels_]為答案 或是 原始資料的答案iris['target'],
# 用該欄位加上顏色判別, 作為額外資訊
# hue = cluster.labels_ 分群結果的答案
# hue=iris["target"]    原始資料的答案
sns.scatterplot(df["sepal length (cm)"],
                df["petal length (cm)"],
                # hue = df["label"])
                hue=iris["target"])


若不知道有幾群? 分群的標準
iris資料有四個欄位(4個維度),所以要降維簡化成 2維
選擇以sepal length 、petal length 作為x axis、y axis
- 衡量親疏程度

朋友圈Ａ<----$\ast 人^{(分到Ａ)}$------------------->朋友圈B
- (假設親密程度 0~10) -->判斷此人有沒有分到對的群
- 1.  10_________________________0__ =>信心程度100%
- 2.  5_________________________5__  =>信心程度50%
- 3.  0_________________________10__ =>信心程度0%
- 4.  6_________________________4__   =>信心程度60%

對於一點$P_i$的親密度

親密度: (越小越好 -> 越近越好)
- 所在群內各個隨機點到 所在群 內其他點的距離的平均的加總
$$
\overbrace{\sum_{i=1}^m \frac{1}{n} \underbrace{\sum_{j=1}^n(P_i - 所在圈_j)^2}_{某一點}}^{圈內全部點} => a
$$
疏遠度: (越大越好 -> 距離越遠越好)
- 所在群內各個隨機點到 最相近的群內 其他點的距離的平均的加總
$$
\sum_{i=1}^m \frac{1}{n} \sum_{j=1}^n(P_i - 最近圈_j)^2 => b 
$$

對照TF-IDF方法 推導 ==>親疏程度

$$
a \times \frac{1}{b}
$$
$$
\begin{cases}
0: a \rightarrow 0 親密程度高,
   b \rightarrow \infty 疏遠程度高 \\
1: a = b \Rightarrow P_i 朋友＝陌生人(距離一樣) \Rightarrow 瞎猜
\end{cases}
$$
$$0 \leq \frac{a}{b} \leq 1$$
$$        好--->瞎$$
$$\color{lime}{\fbox{$0 \leq 1 - \frac{a}{b} \leq 1$}}$$ 
$$        瞎--->好$$
$$       \color{lime}{silhouette \ score}$$


In [33]:
from sklearn.metrics import silhouette_score
for k in range(2, 10):
  test = KMeans(n_clusters=k)
  test.fit(iris["data"])
  print("k:", k)
  # 數學:追求唯一解  統計:給出一些參考的答案
  # 給一個參考的親密程度
  score = silhouette_score(iris["data"], test.labels_)
  print("score", score)
  # 0.5以上是不錯的親疏程度
  # 此例, 分 2群＝0.68 或 3群=0.55 都可以

k: 2
score 0.681046169211746
k: 3
score 0.5528190123564091
k: 4
score 0.4980505049972867
k: 5
score 0.4887488870931048
k: 6
score 0.3648340039670018
k: 7
score 0.3452178646369988
k: 8
score 0.36179003359737993
k: 9
score 0.33549728124663686


文字分群：

應用套件：wordcloud文字雲[https://github.com/amueller/word_cloud]

